“Gradient” approaches : define the brain as a set of continuous scores along manifold axes (≠ parcellisation into distinct areas based on anatomical functions or features)

Core gradient technique : 
1. **computation of an affinity matrix** that captures inter-area similarity of a given feature 
2. application of **dimensionality reduction** method (here diffusion embedding) to identify a gradual ordering of the input matrix in a lower dimensional manifold space.





Ressources 

- https://brainspace.readthedocs.io/en/latest/pages/install.html
- https://brainspace.readthedocs.io/en/latest/pages/getting_started.html

In [1]:
!git clone https://github.com/MICA-MNI/BrainSpace.git
!pip install load_confounds
!pip install vtk
!pip install "nilearn>=0.9.0"

%cd BrainSpace
!python setup.py install

fatal: le chemin de destination 'BrainSpace' existe déjà et n'est pas un répertoire vide.
/home/romy.beaute/projects/hypnomed/BrainSpace/BrainSpace
/home/romy.beaute/anaconda3/lib/python3.9/site-packages/setuptools/dist.py:757: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  warnings.warn(
running install
/home/romy.beaute/anaconda3/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/romy.beaute/anaconda3/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
writing brainspace.egg-info/PKG-INFO
writing dependency_links to brainspace.egg-info/dependency_l

## Import the dataset as timeseries


NB : Timeseries (data) should be a numpy array with the dimensions: nodes x timepoints

In [6]:
import sys, os
import numpy as np
import nibabel as nib
import brainspace
import matplotlib
import sklearn


In [37]:
sys.path.append('/mnt/data/romy/hypnomed/git/diffusion_embedding/scripts_reordered') 

from load_fs import *

sub = 'sub-01'
ses = 'ses-001'
state = 'rs_run-1'

freesurfer_output = '/mnt/data/romy/hypnomed/MRI_raw/BIDS/derivatives/fmriprep-latest/sourcedata/freesurfer'
vol2surf_path = "/mnt/data/romy/hypnomed/git/diffusion_embedding/vol2surf_derivatives/{}/{}".format(sub,ses)
labels = []
timeseries = [None] * 2

for i, h in enumerate(['lh', 'rh']):
    filename = os.path.join(vol2surf_path,'{}_{}_task-{}.fsa5.{}.mgz'.format(sub,ses,state,h))
    timeseries[i] = nib.load(filename.format(h)).get_fdata().squeeze()
    print(timeseries[i].shape) 
    labels.append(nib.freesurfer.read_label(f'{freesurfer_output}/fsaverage5/label/{h}.cortex.label'))
timeseries = np.vstack(timeseries)

print(' - Original shape timeseries : {} --> ie {} nodes x {} timepoints'.format(timeseries.shape,timeseries.shape[0],timeseries.shape[1]))
print(' - Label R + Label L : {} + {} nodes = {} nodes '.format(len(labels[0]),len(labels[1]),len(labels[0])+len(labels[1])))

(10242, 235)
(10242, 235)
 - Original shape timeseries : (20484, 235) --> ie 20484 nodes x 235 timepoints
 - Label R + Label L : 9354 + 9361 nodes = 18715 nodes 


*Compare wih shape from data loaded in tuto*

In [38]:
from brainspace.datasets import fetch_timeseries_preprocessing
ts_tuto = fetch_timeseries_preprocessing()
ts_tuto.shape

(20484, 652)

## Confound regression


Strategy to select and load confounds from fMRIPrep (based on past literature with limited parameters for user customisation).

=> use "simple" preset denoising strategy (as denoising strategy commonly used in resting state functional connectivity)

=> global signal regression : remove confounds without compromising the temporal degrees of freedom.


- https://nilearn.github.io/stable/modules/generated/nilearn.interfaces.fmriprep.load_confounds_strategy.html (COUFOUNDS Nilearn)
- https://nilearn.github.io/stable/glossary.html#term-fMRIPrep (fMRIPREP)

In [62]:
from nilearn.interfaces.fmriprep import load_confounds_strategy
from nilearn import signal

fmriprep_output = '/mnt/data/romy/hypnomed/MRI_raw/BIDS/derivatives/fmriprep-latest/{}/ses-001/func'.format(sub)
fmriprep_simple = '{}_ses-001_task-{}_space-T1w_desc-preproc_bold.nii.gz'.format(sub,state)
fmriprep_aroma = '{}_ses-001_task-{}_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'.format(sub,state)

confounds_out_simple = load_confounds_strategy(os.path.join(fmriprep_output,fmriprep_simple),
                                        denoise_strategy='simple')[0]

print(confounds_out_simple.shape)

confounds_out_simple = load_confounds_strategy(os.path.join(fmriprep_output,fmriprep_simple),
                                        denoise_strategy='simple')[0]

print(confounds_out_simple.shape)


(235, 33)


(20484, 235)

In [67]:
confounds_out_AROMA = load_confounds_strategy(os.path.join(fmriprep_output,fmriprep_file),
                                        denoise_strategy='ica_aroma')

ValueError: Input must be ~desc-smoothAROMAnonaggr_bold for full ICA-AROMA strategy.

In [63]:
confounds_out

,cosine00,cosine01,cosine02,cosine03,cosine04,cosine05,cosine06,csf,rot_x,rot_x_derivative1,...,trans_x_power2,trans_y,trans_y_derivative1,trans_y_derivative1_power2,trans_y_power2,trans_z,trans_z_derivative1,trans_z_derivative1_power2,trans_z_power2,white_matter
0,0.092251,0.092245,0.092235,0.092220,0.092202,0.092179,0.092152,211.381513,0.023669,0.000056,...,-0.111850,-0.046145,0.024992,0.000136,0.002945,0.050773,0.040857,0.000374,-0.020487,41.268159
1,0.092235,0.092179,0.092086,0.091957,0.091790,0.091586,0.091346,54.460432,0.023546,0.000056,...,-0.111438,-0.020865,0.024992,0.000136,-0.000085,0.090898,0.040857,0.000374,-0.024370,41.299174
2,0.092202,0.092047,0.091790,0.091430,0.090968,0.090405,0.089740,72.657765,0.023348,-0.000020,...,-0.111452,-0.014237,0.006341,-0.000459,-0.000668,0.070590,-0.019578,-0.000824,-0.022807,59.523952
3,0.092152,0.091849,0.091346,0.090642,0.089740,0.088642,0.087349,179.543399,0.023344,0.000175,...,-0.111848,-0.046187,-0.032238,0.000518,0.002951,0.049595,-0.020264,-0.000796,-0.020324,66.407435
4,0.092086,0.091586,0.090755,0.089595,0.088111,0.086309,0.084194,105.361708,0.023016,-0.000150,...,-0.111848,-0.000966,0.044933,0.001542,-0.001571,0.080438,0.031574,-0.000285,-0.023668,68.429112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,-0.092086,0.091586,-0.090755,0.089595,-0.088111,0.086309,-0.084194,-29.715734,-0.017700,-0.000052,...,0.118527,-0.006660,0.017184,-0.000198,-0.001227,-0.160498,0.064027,0.002770,0.053073,-34.161428
231,-0.092152,0.091849,-0.091346,0.090642,-0.089740,0.088642,-0.087349,-66.521009,-0.017884,-0.000006,...,0.118599,0.009142,0.015515,-0.000254,-0.002023,-0.121964,0.039265,0.000248,0.033000,-42.770629
232,-0.092202,0.092047,-0.091790,0.091430,-0.090968,0.090405,-0.089740,-99.798139,-0.017373,0.000690,...,0.107646,-0.023645,-0.033075,0.000572,0.000186,-0.190593,-0.067898,0.003473,0.070815,-44.753763
233,-0.092235,0.092179,-0.092086,0.091957,-0.091790,0.091586,-0.091346,-72.792664,-0.017844,-0.000292,...,0.094375,-0.019612,0.003745,-0.000487,-0.000202,-0.208576,-0.017252,-0.000913,0.082281,-42.383331


In [64]:
# Confound regression
clean_ts = signal.clean(timeseries.T, confounds=confounds_out).T
clean_ts.shape


(20484, 235)

*Compare wih shape from data loaded in tuto*

In [53]:
from brainspace.datasets import load_confounds_preprocessing
confounds_out_tuto = load_confounds_preprocessing()
confounds_out_tuto.shape

(652, 29)

In [ ]:
label = nib.freesurfer.read_label(f'{freesurfer_output}/fsaverage5/label/{h}.cortex.label')

(20484, 652)

In [11]:
from load_fs import load_fs


In [13]:
data = load_fs(sub,ses,state)

In [14]:
data.shape

(18715, 235)

In [16]:
ts = []
label = []
for h in ['lh','rh']: #order of hemisphere is important !
    #loop through hemispheres
    ts.append(nib.load(f'{vol2surf_path}/{sub}_{ses}_task-{state}.fsa5.{h}.mgz').get_data().squeeze())


/tmp/ipykernel_2164287/783497625.py:5: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  ts.append(nib.load(f'{vol2surf_path}/{sub}_{ses}_task-{state}.fsa5.{h}.mgz').get_data().squeeze())


In [18]:
ts

[array([[10932.243 , 10960.968 , 10919.1455, ..., 10996.024 , 11006.026 ,
         11029.652 ],
        [13640.841 , 13594.97  , 13698.012 , ..., 13712.899 , 13753.704 ,
         13721.796 ],
        [12543.235 , 12536.526 , 12554.685 , ..., 12716.575 , 12758.684 ,
         12787.699 ],
        ...,
        [11553.502 , 11637.631 , 11475.968 , ..., 11605.24  , 11555.416 ,
         11648.314 ],
        [11577.567 , 11679.924 , 11489.445 , ..., 11633.264 , 11574.406 ,
         11651.901 ],
        [11756.006 , 11855.627 , 11680.64  , ..., 11803.974 , 11768.624 ,
         11833.498 ]], dtype=float32),
 array([[13043.551 , 12975.011 , 13040.363 , ..., 12932.149 , 12987.742 ,
         13013.312 ],
        [13604.597 , 13532.259 , 13599.715 , ..., 13576.8545, 13536.136 ,
         13543.137 ],
        [13175.862 , 13153.214 , 13211.118 , ..., 13286.922 , 13402.007 ,
         13487.886 ],
        ...,
        [ 2425.0957,  2413.5315,  2436.7954, ...,  2399.8787,  2366.5193,
          2402.7522